## Imports and Installs

In [ ]:
!pip install contractions
!pip install emoji
!pip install ekphrasis

import math
import pandas as pd
import re
import emoji
import nltk
import contractions
import torch

nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer 
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize 
from nltk.tokenize.treebank import TreebankWordDetokenizer
import requests 
from pprint import pprint
import numpy as np
from nltk.stem import PorterStemmer

import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras

import os
import pprint
import json
import random
import string
import sys
from ekphrasis.classes.segmenter import Segmenter
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Load file

In [ ]:
df_comb = pd.read_csv('/content/drive/My Drive/Labelled (Binary - previous day open close)/labelled_WMT.csv')
combine_ds = df_comb.sample(frac=1)
combine_ds

## Preprocess data

Fill NaN messages with '0', which will be dropped later


In [ ]:
combine_ds.isna().values.any()
combine_ds['message'] = combine_ds['message'].fillna('0')

Case-folding


In [ ]:
combine_ds['message'] = combine_ds['message'].str.lower()
message = combine_ds['message'].tolist()
print(message[:10])

Functions definitions

In [ ]:
# Function to remove stopwords
def remove_stopwords(msg):
    filtered_sentence = [w for w in msg_tokens if not w in stop_words]
    return filtered_sentence

# Function to remove punctuations, URLs, Mentions, Hashtags and _ from emojis
def remove_punctuation_re(x):
    x = ' '.join(re.sub("https?://\S+","",x).split())     # Removing URLs
    x = ' '.join(re.sub("^@\S+|\s@\S+","",x).split())     # Removing Mentions
    x = ' '.join(re.sub(r'[^\w\s]'," ",x).split())        # Removes Hashtags
    x = ' '.join(re.sub(r'_'," ",x).split())              # Removing _ from emojis text
    return x

# Function to replace repeating letter
def rpt_replace(match):
    return match.group(1)+match.group(1)

# Function to substitute original word with replaced word, if any
def processRepeatings(data):
    re_t= re.sub(message_rpt, rpt_replace, data )
    return re_t

In [ ]:
# Download stopwords
stop_words = sw.words("english")

# Create tokenizer and detokenizer
tweet_tokenizer = TweetTokenizer()
detokenizer = TreebankWordDetokenizer()

# For repeating characters in words
message_rpt = re.compile(r"(.)\1{2,}", re.IGNORECASE)

# Segmenter using the word statistics from Twitter
seg_tw = Segmenter(corpus="twitter")

In [ ]:
message_p = []
count = 0

for msg in message:
    print(count)
    count += 1

    if msg == '0': # NaN replaced by '0'
      message_p.append('-1')

    else:
      # remove emojis
      msg = emoji.demojize(msg)

      # fix contractions
      msg = contractions.fix(msg)

      # remove punctuations
      msg = remove_punctuation_re(msg) 

      # tokenize
      msg_tokens = tweet_tokenizer.tokenize(msg)

      # For compressing elongated words using Word segmenter
      message_seg = []
      for w in msg_tokens:
        if len(w) >= 300: # truncate word to 100 characters if it's original length is more than 300
          w = w[:100]
          print(w)
        message_seg.append(seg_tw.segment(w))

      # remove stopwords
      msg = remove_stopwords(message_seg)

      if 'rt' in msg:
        # remove retweets
        message_p.append('-1')
      else: 
        # detokenize
        msg = detokenizer.detokenize(msg)

        # removing repeating characters like hurrrryyyyyy-- worrks on tokenized list
        msg = processRepeatings(msg)

        message_p.append(msg)


In [ ]:
message_p[:10]

Assign preprocessed message list to dataframe


In [ ]:
combine_ds['message'] = message_p
combine_ds

In [ ]:
combine_ds['polarity'].value_counts()

Drop Retweets


In [ ]:
combine_ds.drop(combine_ds[combine_ds['message'] == '-1'].index, inplace = True) 
combine_ds

In [ ]:
combine_ds['polarity'].value_counts()

## Special handling for FB and TSLA

For FB, drop tweets before IPO date


In [ ]:
# FB_IPO = '2012-05-21'
# combine_ds.drop(combine_ds[combine_ds['Date'] < '2012-05-21'].index, inplace = True) 
# combine_ds

For TSLA, drop tweets before IPO date


In [ ]:
# TSLA_IPO = '2010-06-30'
# combine_ds.drop(combine_ds[combine_ds['Date'] < TSLA_IPO].index, inplace = True) 
# combine_ds

In [ ]:
combine_ds['polarity'].value_counts()

## Save preprocessed company file 

In [ ]:
combine_ds.to_csv('labelled_WMT.csv')
!cp labelled_WMT.csv "/content/drive/My Drive/Prep Labelled (binary - previous day)"